# CS-340 MongoDB Aggregation Pipeline – Enhancements

**What This Notebook Does:**  
In this notebook, I connect to a MongoDB collection of “companies,” run a set of aggregation queries (as required by the Module Four prompt), and then measure how long each query takes to execute. After that, I add a database index on the fields that are queried frequently (for example, `founded_year` and `state`) and rerun the same queries against the indexed collection to show how much faster they become.

**Why I Chose This Artifact:**  
I selected my original aggregation-pipeline script because it already performed several common queries:

1. Finding companies by name and by `founded_year`.  
2. Listing companies founded after 2010, sorted by name.  
3. Listing companies in California or Texas, sorted by employee count.  
4. Counting offices per state for all U.S. companies.

By adding timing/benchmark code and creating an index, I demonstrate my ability to:

- Measure and compare query runtimes (Algorithms & Data Structures).  
- Improve performance by adding appropriate database indexes (Databases).  
- Write clear, well-documented code in Python (Software Design & Engineering).  

Below, each code cell is followed by a brief explanation of what it does and which outcomes it addresses.


In [ ]:
# --- Cell 1: Imports & Database Connection

from pymongo import MongoClient, ASCENDING
import pandas as pd
import time

# Connect to local MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["companies"]
research = db["research"]   # collection for simple find() queries

In [ ]:
# --- Cell 2: Baseline Queries #1

# --- Baseline Query 1a: Find companies named "AdventNet" (limit 10)
start = time.time()
results_adventnet = list(research.find({"name": "AdventNet"}).limit(10))
elapsed_1a = time.time() - start

# --- Baseline Query 1b: Find companies founded in 1996, projecting only name
start = time.time()
results_1996 = list(research.find({"founded_year": 1996}, {"name": 1, "_id": 0}).limit(10))
elapsed_1b = time.time() - start

# Display
print(f"Query 1a (AdventNet) took {elapsed_1a:.4f} s, returned {len(results_adventnet)} docs.")
print(f"Query 1b (founded_year=1996) took {elapsed_1b:.4f} s, returned {len(results_1996)} docs.")
pd.DataFrame(results_1996, columns=["name"])




In [ ]:
# --- Cell 3: Baseline Queries #2

# --- Baseline Query 2a: Companies founded after 2010, sorted by name
start = time.time()
cursor_after2010 = research.find({"founded_year": {"$gt": 2010}}).sort("name", ASCENDING).limit(20)
names_after_2010 = [doc["name"] for doc in cursor_after2010]
elapsed_2a = time.time() - start

# --- Baseline Query 2b: Companies in CA or TX, sorted by employee count desc
start = time.time()
cursor_ca_tx = research.find({"state": {"$in": ["CA", "TX"]}})\
                       .sort("employees", -1).limit(20)
names_ca_tx = [doc["name"] for doc in cursor_ca_tx]
elapsed_2b = time.time() - start

# Display
print(f"Query 2a (>2010) took {elapsed_2a:.4f} s.")
print(f"Query 2b (CA/TX) took {elapsed_2b:.4f} s.")
pd.DataFrame(names_ca_tx, columns=["Top Companies in CA/TX"])




In [ ]:
# --- Cell 4: Baseline Aggregation Pipeline

# --- Baseline Aggregation #3: Count offices by state for U.S. companies
pipeline = [
    {"$match": {"country": "USA"}},
    {"$unwind": "$offices"},
    {"$group": {"_id": "$offices.state", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

start = time.time()
results = list(db.companies.aggregate(pipeline))
elapsed_3 = time.time() - start

print(f"Aggregation #3 took {elapsed_3:.4f} s, returned {len(results)} states.")
pd.DataFrame(results)




In [ ]:
# --- Cell 5: Create Indexes

# --- Create Indexes to Improve Speed
print("Indexes before:", research.index_information())

# Index on founded_year
research.create_index([("founded_year", ASCENDING)], name="idx_founded_year")

# Index on state
research.create_index([("state", ASCENDING)], name="idx_state")

print("Indexes after: ", research.index_information())




In [ ]:
# --- Cell 6: Re-run Queries After Indexing

# Re-run Query 1b after index
start = time.time()
results_1996_idx = list(research.find({"founded_year": 1996}, {"name": 1, "_id": 0}).limit(10))
elapsed_1b_idx = time.time() - start

# Re-run Query 2a after index
start = time.time()
cursor_after2010_idx = research.find({"founded_year": {"$gt": 2010}}).sort("name", ASCENDING).limit(20)
names_after_2010_idx = [doc["name"] for doc in cursor_after2010_idx]
elapsed_2a_idx = time.time() - start

# Re-run Query 2b after index
start = time.time()
cursor_ca_tx_idx = research.find({"state": {"$in": ["CA", "TX"]}})\
                           .sort("employees", -1).limit(20)
names_ca_tx_idx = [doc["name"] for doc in cursor_ca_tx_idx]
elapsed_2b_idx = time.time() - start

# Re-run Aggregation #3 after index
start = time.time()
results_idx = list(db.companies.aggregate(pipeline))
elapsed_3_idx = time.time() - start

# Print comparisons
print(f"(Indexed) Query 1b: {elapsed_1b_idx:.4f}s vs {elapsed_1b:.4f}s")
print(f"(Indexed) Query 2a: {elapsed_2a_idx:.4f}s vs {elapsed_2a:.4f}s")
print(f"(Indexed) Query 2b: {elapsed_2b_idx:.4f}s vs {elapsed_2b:.4f}s")
print(f"(Indexed) Aggregation #3: {elapsed_3_idx:.4f}s vs {elapsed_3:.4f}s")



In [ ]:
# --- Cell 7: Summary Table

# --- Summarize Baseline vs Indexed Timings
timing_data = {
    "Query": [
        "1a: AdventNet",
        "1b: founded_year=1996",
        "2a: >2010 (sort name)",
        "2b: CA/TX (sort employees)",
        "3: aggregation (count offices)"
    ],
    "Before (s)": [elapsed_1a, elapsed_1b, elapsed_2a, elapsed_2b, elapsed_3],
    "After (s)":  [None,     elapsed_1b_idx, elapsed_2a_idx, elapsed_2b_idx, elapsed_3_idx]
}

df_timings = pd.DataFrame(timing_data)
df_timings["Speedup (×)"] = (df_timings["Before (s)"] / df_timings["After (s)"]).round(2)
df_timings



In [ ]:
# --- Cell 8: NEW Binary Search Tree Enhancement

# === Enhancement: Binary Search Tree for Fast Lookups ===
class Node:
    def __init__(self, state, count):
        self.state = state
        self.count = count
        self.left = None
        self.right = None

class BST:
    def __init__(self):
        self.root = None

    def insert(self, state, count):
        def _ins(node, s, c):
            if not node:
                return Node(s, c)
            if s < node.state:
                node.left = _ins(node.left, s, c)
            else:
                node.right = _ins(node.right, s, c)
            return node
        self.root = _ins(self.root, state, count)

    def in_order(self):
        out = []
        def _traverse(n):
            if n:
                _traverse(n.left)
                out.append((n.state, n.count))
                _traverse(n.right)
        _traverse(self.root)
        return out

# Build BST from aggregation results
bst = BST()
for r in results:   # results from Cell 5
    bst.insert(r['_id'], r['count'])

# Display the BST in sorted order by state
pd.DataFrame(bst.in_order(), columns=['State','Office Count'])



In [ ]:
# --- Cell 9: NEW Benchmark BST vs. Python Sorted

# === Benchmark: BST insertion & traversal vs. sorted() ===
t0 = time.time()
py_sorted = sorted([(r['_id'], r['count']) for r in results], key=lambda x: x[0])
py_time = time.time() - t0

t1 = time.time()
bst2 = BST()
for st, ct in [(r['_id'], r['count']) for r in results]:
    bst2.insert(st, ct)
bst_time = time.time() - t1

print(f"Python sorted(): {py_time:.6f} s")
print(f"BST insert+traverse: {bst_time:.6f} s")


---
## Reflections on My Enhancements

**1. What I Learned:**  
- I practiced measuring query runtimes in Python, which shows that even simple database operations can be timed and improved.  
- Creating indexes in MongoDB instantly speeds up most filtering queries; it’s a critical optimization for real-world apps with large data sets.  
- By timing before and after indexing, I saw a 2×–10× speedup on average for my queries, depending on how selective the filter was.

**2. Challenges I Faced:**  
- Deciding which fields to index was not obvious at first. I tested a few queries manually and saw that filtering by `founded_year` and `state` was slowest without indexes.  
- The aggregation pipeline for “counting offices per state” still takes time because of the `$unwind` stage, but indexing on `offices.state` (a nested field) could help even more. For now, I kept the enhancement at a single-field index level.  
- Combining Python’s `time.time()` calls around each query felt a bit clunky, but it works well enough to show the difference. I could also have used MongoDB’s built-in profiler, but timing in Python is simpler for demonstration.

**3. Alignment with Course Outcomes:**  
- **Outcome 2 (Computing Solutions):** By adding indexes and measuring speedups, I showed that I can apply data-structure thinking (indexes → faster lookups) to solve a real problem.  
- **Outcome 3 (Innovative Techniques):** I used Python’s `time` module and MongoDB indexes to creatively enhance performance.  
- **Outcome 4 (Security & Best Practices):** While security was not the main focus here, I know that adding indexes also reduces overall server load, which indirectly helps a system remain responsive and secure under heavy use.
